In [1]:
from datasets import load_dataset
from gensim.utils import tokenize
from tqdm import tqdm
from textblob import TextBlob
from collections import Counter
from torch.optim import AdamW
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
data=load_dataset('imdb')
device='cpu'
from torch.nn import Module
from torch import nn 

/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_sentences=[]
sent_threshold=32

for text_block in tqdm(data['train']['text']):
    for sentences in TextBlob(text_block).sentences:
        len_of_sent=sentences.words 
        for sent in sentences.split('.<br /><br />'):
            if len(len_of_sent)<sent_threshold:
                all_sentences.append(sent)
    
    


100%|███████████████████████████████████████████████████████████| 25000/25000 [00:24<00:00, 1019.94it/s]


In [3]:
words=[]
for sent in tqdm(all_sentences):
    for word in tokenize(sent):
        words.append(word.lower())

100%|███████████████████████████████████████████████████████| 227434/227434 [00:01<00:00, 125180.27it/s]


In [4]:
cntr=Counter(words)

In [5]:
vocab_size=40000
our_words=set(['<unk>','<bos>','<eos>','<pad>'])
for word,cnt in cntr.most_common()[0:vocab_size]:
    our_words.add(word)

In [6]:
len(our_words)

40004

In [7]:
for i in tokenize(all_sentences[5]):
    print(i)

I
Am
Curious
Yellow
is
a
risible
and
pretentious
steaming
pile


In [8]:
word2int={word:cnt for cnt,word in enumerate(our_words)}
int2word={cnt:word for cnt,word in enumerate(our_words)}

In [9]:
class word_dataset(Dataset):
    def __init__(self,data):
        super(word_dataset,self).__init__()
        self.data=data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        selected_sent=self.data[idx]
        indexed_text=[word2int['<bos>']]
        main_text=[word2int[word] if word in word2int else word2int['<unk>'] for word in tokenize(selected_sent.lower())]
        indexed_text=indexed_text+main_text+[word2int['<eos>']]
        return indexed_text
        
        
        

In [10]:
def make_batch(batch):
    lenghts=(len(x) for x in batch)
    max_len=max(lenghts)

    new_batch=[]
    for sent in batch:
        for pad in range(max_len-len(sent)):
            sent.append(word2int['<pad>'])
        new_batch.append(sent)

    new_batch=torch.LongTensor(new_batch).to(device)

    

    return {
        'inp':new_batch[:,:-1],
        'label':new_batch[:,1:]
    }
    

In [11]:
train_data=word_dataset(all_sentences)
train_dataloader=DataLoader(train_data,batch_size=32,shuffle=True,collate_fn=make_batch)

In [30]:
class LanguageModel(Module):
    def __init__(self,vocab_size,hidden_size):
        super().__init__()

        self.emb=nn.Embedding(vocab_size,hidden_size)
        self.gru=nn.GRU(hidden_size,hidden_size,num_layers=3,batch_first=True)

        self.lay_with_drop=nn.Sequential(
            nn.Tanh(),
            nn.Linear(hidden_size,hidden_size),
            nn.Dropout(),
            nn.Tanh()
        )

        self.final_lin=nn.Sequential(
            nn.Linear(hidden_size,vocab_size)
        )

        
    def forward(self,text):
        
        emb_x=self.emb(text)
        
    
        x,_=self.gru(emb_x)

        agr_x=x.mean(dim=1)#???????
        

        x=self.lay_with_drop(agr_x)
        x=self.final_lin(x)
        print(x.shape)
        return x
        

        

In [31]:
model=LanguageModel(len(our_words),128)
loss_fn=nn.CrossEntropyLoss(ignore_index=word2int['<pad>'])
optimizer=AdamW(model.parameters())

In [32]:
len(our_words)

40004

In [33]:
def train(model,loss_fn,optimizer,dataloader):
    for batch in dataloader:
        pred=model(batch['inp']).flatten(start_dim=0,end_dim=1)
        print(pred.shape,batch['label'].flatten().shape)
        loss=loss_fn(pred,batch['label'].flatten())

In [34]:
train(model,loss_fn,optimizer,train_dataloader)

torch.Size([32, 40004])
torch.Size([1280128]) torch.Size([1024])


RuntimeError: size mismatch (got input: [1280128], target: [1024])